# STA Ellipse : Visualization tips

Hi, 

One of the main objective of the pipeline is the prediction of the receptive field ellipse. In this notebook, we will give some tips on:
- how to retrieve the data corresponding to the RF
- how to use it for images with different resolutions (for instance STA and stimulus) 
- how to zoom on the receptive field

In [ ]:
import matplotlib.pyplot as plt 
import os
import skimage.io as sk
import sys
sys.path.append('..')
from  utils import *

## Loading the images and receptive field

First, let's load an example stimulus image (usually an png file, a frame from a bin/vec file or a checkerboard matrix).

In [ ]:
# Load the stimulus images
stimulus_path = r'../docs/image_2997.png'
stimulus = sk.imread(stimulus_path, as_gray=True) / 254

plt.imshow(stimulus)

The STA and the RF ellipse can be loaded from the sta_data_3D_fitted.pkl created by the notebook 2 of the pipeline.

In [ ]:
# Load example .pkl file with only one cell with id 0
sta_results_ex = np.load(r'../docs/sta_data_3D_fitted_example.pkl', allow_pickle=True)
cell_id = 0

# Load receptive field parameters
rf_parameters = sta_results_ex[cell_id]["center_analyse"]["EllipseCoor"]

# Load spatial STA
spatial_sta = sta_results_ex[cell_id]["center_analyse"]["Spatial"];

To plot the STA and the RF, you can use the function plot_sta from the pipeline.

In [ ]:
fig, ax = plt.subplots()
plot_sta(ax, spatial_sta, rf_parameters)
ax.set_yticks([])
ax.set_xticks([]);

However, this function is not suited for any other visualization.

## Plot the receptive field on the stimulus image (or any image with a different resolution than the STA)

First, let's look at  *rf_parameters* :

In [ ]:
print(f"""
      rf_parameters[0] : {rf_parameters[0]} (height of the gaussian)
      rf_parameters[1] : {rf_parameters[1]} (x center of the gaussian)
      rf_parameters[2] : {rf_parameters[2]} (y center of the gaussian)
      rf_parameters[3] : {rf_parameters[3]} (sigma along x of the gaussian)
      rf_parameters[4] : {rf_parameters[4]} (sigma along y of the gaussian)
      rf_parameters[5] : {rf_parameters[5]} (angle of the 2D gaussian)
      """)



The *rf_parameters* object is computed with the resolution on the checkerboard used to detect STA (for instance 72x72 checks). Hence, we need to transform this object to fit the resolution of the stimulus (fore example a 864*864 natural image). Because matplotlib image coordinates range from -0.5 to image size -0.5 (ex: 863.5), we need to apply the following transformation to the center coordinates of the ellipse (rf_parameters[1], rf_parameters[2]): 

    new_x = (old_x + 0.5) * (new_resolution / old/resolution) - 0.5

For instance, in our example, the transformation is :

    new_x = (old_x + 0.5) * (864/72) - 0.5

We will also need to scale the x and y sigma by a (new_resolution / old_resolution) factor.


In [ ]:
stimulus_rf_parameters = [
    rf_parameters[0],
    (rf_parameters[1]+0.5) * 12 - 0.5,
    (rf_parameters[2]+0.5) * 12 - 0.5,
    rf_parameters[3] * 12,
    rf_parameters[4] * 12,
    rf_parameters[5],
]

stimulus_rf_parameters

Now, we can plug in the new parameters into the gaussian2D object of the pipeline and plot the ellipse on the stimulus with matplotlib.

In [ ]:
gaussian = gaussian2D((864,864), *stimulus_rf_parameters)

level_factor = 0.32 # 1 sigma

fig, ax = plt.subplots()
im = ax.imshow(stimulus)  # , interpolation="gaussian")
ax.contour(
    np.abs(gaussian),
    levels=[level_factor * np.max(np.abs(gaussian))],
    colors="chartreuse", # For sure the best color
    linestyles="solid",
    alpha=0.8,
)
ax.set_xticks([])
ax.set_yticks([]);

*Some optional details :*
- *gaussian2D : The main role of this object is to build the 2D gaussian on a space of the given size (here 864x864).*
- *ax.contour : A matplotlib function to plot 2D gaussian. The levels parameters can be used to select the height at which to cut the gaussian (0.32 = 1 sigma, 0.05 = 2 sigma).*

Not that we can use the same strategy to reproduce the plot_sta function, which will be useful to zoom on the STA.

In [ ]:
gaussian = gaussian2D((72,72), *rf_parameters) # note the difference in the size of the space

level_factor = 0.32 # 1 sigma

fig, ax = plt.subplots()
im = ax.imshow(spatial_sta)
ax.contour(
    np.abs(gaussian),
    levels=[level_factor * np.max(np.abs(gaussian))],
    colors="chartreuse",
    linestyles="solid",
    alpha=0.8,
)
ax.set_xticks([])
ax.set_yticks([]);

## Zoom on the receptive field

Now that we have already converted the rf_parameters object to fit the stimulus as well, this part should be quite easy. We will simply use the function set_xlim/ser_ylim of matplotlib. Keep in mind that matplotlib images are encoded like matrices (y axis go downward). And remember to adapt your zoom (in number of pixel around the center of the ellipse) to the resolution of the picture.

In [ ]:
zoom_level = 5 # pixels around the center

# 1. Zoom on the STA

gaussian = gaussian2D((72,72), *rf_parameters) # note the difference in the size of the space

ax = plt.subplot(1,2,1)
im = ax.imshow(spatial_sta)
ax.contour(
    np.abs(gaussian),
    levels=[level_factor * np.max(np.abs(gaussian))],
    colors="chartreuse",
    linestyles="solid",
    alpha=0.8,
)
ax.set_xticks([])
ax.set_yticks([])
ax.set_xlim([rf_parameters[1] - zoom_level, rf_parameters[1] + zoom_level])
ax.set_ylim([rf_parameters[2] + zoom_level, rf_parameters[2] - zoom_level]) # y axis go downward

# 2. Zoom on the Stimulus with the same zoom

gaussian = gaussian2D((864,864), *stimulus_rf_parameters)

level_factor = 0.32 # 1 sigma

ax = plt.subplot(1,2,2)
im = ax.imshow(stimulus)  # , interpolation="gaussian")
ax.contour(
    np.abs(gaussian),
    levels=[level_factor * np.max(np.abs(gaussian))],
    colors="chartreuse", # For sure the best color
    linestyles="solid",
    alpha=0.8,
)
ax.set_xticks([])
ax.set_yticks([]);
ax.set_xlim([stimulus_rf_parameters[1] - zoom_level*12, stimulus_rf_parameters[1] + zoom_level*12])
ax.set_ylim([stimulus_rf_parameters[2] + zoom_level*12, stimulus_rf_parameters[2] - zoom_level*12]);

Finally, here are a few additional steps you can go through depending on our needs :
- sharpen and smooth the STA
- add grids on top of the images (useful to check everything is right as well)

In [ ]:
zoom_level = 5 # pixels around the center

# 1. Zoom on the STA

gaussian = gaussian2D((72,72), *rf_parameters) # note the difference in the size of the space

ax = plt.subplot(1,2,1)

spatial = spatial_sta**2 * np.sign(spatial_sta) # sharpen
cmap = "RdBu_r" # Change colors
im = ax.imshow(spatial_sta, cmap=cmap, interpolation="gaussian")  # , interpolation="gaussian")
ax.contour(
    np.abs(gaussian),
    levels=[level_factor * np.max(np.abs(gaussian))],
    colors="chartreuse",
    linestyles="solid",
    alpha=0.8,
)

# Let's add a 24*24 gridline
for k in range(24):
    ax.hlines(
        y=3 * k - 0.5,
        xmin=-0.5,
        xmax=71.5,
        colors="silver",
        linestyles="--",
        alpha=0.8,
    )
    ax.vlines(
        x=3 * k - 0.5,
        ymin=-0.5,
        ymax=71.5,
        colors="silver",
        linestyles="--",
        alpha=0.8,
    )
                
ax.set_xticks([])
ax.set_yticks([])
ax.set_xlim([rf_parameters[1] - zoom_level, rf_parameters[1] + zoom_level])
ax.set_ylim([rf_parameters[2] + zoom_level, rf_parameters[2] - zoom_level])

# 2. Zoom on the Stimulus with the same zoom

gaussian = gaussian2D((864,864), *stimulus_rf_parameters)

level_factor = 0.32 # 1 sigma

ax = plt.subplot(1,2,2)
im = ax.imshow(stimulus)  # , interpolation="gaussian")
ax.contour(
    np.abs(gaussian),
    levels=[level_factor * np.max(np.abs(gaussian))],
    colors="chartreuse", # For sure the best color
    linestyles="solid",
    alpha=0.8,
)

# 24*24 grid lines (note once again the difference in resolution and how matplotlib coordinates start at -0.5)
for k in range(24): 
    ax.hlines(
        y=36 * k - 0.5,
        xmin=-0.5,
        xmax=863.5,
        colors="silver",
        linestyles="--",
    )
    ax.vlines(
        x=36 * k - 0.5,
        ymin=-0.5,
        ymax=863.5,
        colors="silver",
        linestyles="--",
    )
                
ax.set_xticks([])
ax.set_yticks([]);
ax.set_xlim([stimulus_rf_parameters[1] - zoom_level*12, stimulus_rf_parameters[1] + zoom_level*12])
ax.set_ylim([stimulus_rf_parameters[2] + zoom_level*12, stimulus_rf_parameters[2] - zoom_level*12]);

As a conclusion, be careful when you use the ellipse object of the pipeline as you might easily misplace the RF on the stimulus if you are not careful enough.

Thanks for reading !

Baptiste Lorenzi (lorenzibaptiste4@gmail.com)